# Run MariaDB with Docker

In [1]:
!docker run --name mariadb-11.7 \
-e MYSQL_ROOT_PASSWORD=12345 \
-e MYSQL_DATABASE=properties_db \
-p 3306:3306 \
-d mariadb:11.7-rc


4455a02fb0dfab39c97a0529f35a0ced4a2b6e1575cf6b13361973e3b80a2056


In [2]:
# Import required libraries
import mysql.connector
from mysql.connector import Error

# Database connection configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '12345',
    'database': 'properties_db',
    'port': 3306,
    'charset': 'utf8mb4',
    'collation': 'utf8mb4_general_ci'  # MariaDB compatible collation
}

# Function to create database connection
def create_connection():
    connection = None
    try:
        connection = mysql.connector.connect(**db_config)
        if connection.is_connected():
            print("Successfully connected to MariaDB")
            print(f"Connected to Server version {connection.get_server_info()}")
            return connection
    except Error as e:
        print(f"Error connecting to MariaDB: {e}")
        print("\nConnection details (excluding password):")
        safe_config = {k:v for k,v in db_config.items() if k != 'password'}
        print(safe_config)
        return None

# Create table
def create_table(connection):
    cursor = None
    if not connection:
        print("No valid connection provided")
        return
        
    try:
        cursor = connection.cursor()
        create_table_query = """
        CREATE TABLE IF NOT EXISTS properties (
            id INT AUTO_INCREMENT PRIMARY KEY,
            property_name VARCHAR(100) NOT NULL,
            address VARCHAR(200) NOT NULL,
            price DECIMAL(10,2) NOT NULL,
            bedrooms INT,
            bathrooms INT,
            square_feet DECIMAL(8,2),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        ) CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci
        """
        cursor.execute(create_table_query)
        connection.commit()
        print("Table created successfully")
    except Error as e:
        print(f"Error creating table: {e}")
    finally:
        if cursor:
            cursor.close()

# Insert data into table
def insert_property(connection, property_data):
    cursor = None
    if not connection:
        print("No valid connection provided")
        return
        
    try:
        cursor = connection.cursor()
        insert_query = """
        INSERT INTO properties 
        (property_name, address, price, bedrooms, bathrooms, square_feet)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, property_data)
        connection.commit()
        print("Data inserted successfully")
    except Error as e:
        print(f"Error inserting data: {e}")
    finally:
        if cursor:
            cursor.close()

# Function to delete table
def delete_table(connection):
    cursor = None
    if not connection:
        print("No valid connection provided")
        return
        
    try:
        cursor = connection.cursor()
        drop_table_query = "DROP TABLE IF EXISTS properties"
        cursor.execute(drop_table_query)
        connection.commit()
        print("Table 'properties' deleted successfully")
    except Error as e:
        print(f"Error deleting table: {e}")
    finally:
        if cursor:
            cursor.close()

# Function to delete database
def delete_database(connection):
    cursor = None
    if not connection:
        print("No valid connection provided")
        return
        
    try:
        cursor = connection.cursor()
        # First, switch to a different database to be able to drop the current one
        cursor.execute("USE mysql")
        drop_db_query = "DROP DATABASE IF EXISTS properties_db"
        cursor.execute(drop_db_query)
        connection.commit()
        print("Database 'properties_db' deleted successfully")
    except Error as e:
        print(f"Error deleting database: {e}")
    finally:
        if cursor:
            cursor.close()
            
# Function to test database connection and show databases
def test_connection():
    try:
        # First try to connect without specifying database
        test_config = db_config.copy()
        test_config.pop('database', None)
        
        conn = mysql.connector.connect(**test_config)
        if conn.is_connected():
            cursor = conn.cursor()
            cursor.execute("SHOW DATABASES")
            databases = cursor.fetchall()
            print("\nAvailable databases:")
            for db in databases:
                print(f"- {db[0]}")
            cursor.close()
            conn.close()
    except Error as e:
        print(f"Error during connection test: {e}")

In [3]:

# Usage example 2: Database setup and data insertion
# Step 1: Test connection and list available databases
print("Testing database connection...")
test_connection()

# Step 2: Connect to specific database
print("\nAttempting to connect to specific database...")
connection = create_connection()

if connection:
    # Step 3: Create new table
    print("\nCreating table...")
    create_table(connection)
    
    # Step 4: Insert sample property data
    print("\nInserting sample data...")
    sample_property = ('Sunset Villa', '123 Ocean Drive', 450000.00, 3, 2, 2000.50)
    insert_property(connection, sample_property)
    
    # Step 5: Clean up by closing connection
    connection.close()
    print("\nDatabase connection closed")
else:
    print("\nFailed to establish database connection. Please check your configuration and ensure the database exists.")

Testing database connection...

Available databases:
- information_schema
- mysql
- performance_schema
- properties_db
- sys

Attempting to connect to specific database...
Successfully connected to MariaDB
Connected to Server version 11.7.1-MariaDB-ubu2404

Creating table...
Table created successfully

Inserting sample data...
Data inserted successfully

Database connection closed


In [4]:
# Usage example 1: Database and table cleanup
# Step 1: Test connection and establish initial connection
print("Connecting to database...")
connection = mysql.connector.connect(**db_config)

if connection.is_connected():
    print("Successfully connected to MariaDB")
    
    # Step 2: Clean up existing table
    print("\nDeleting table...")
    delete_table(connection)
    
    # Step 3: Clean up existing database
    print("\nDeleting database...")
    delete_database(connection)
    
    # Step 4: Clean up by closing connection
    connection.close()
    print("\nDatabase connection closed")
else:
    print("\nFailed to connect to database. Please check your configuration.")

Connecting to database...
Successfully connected to MariaDB

Deleting table...
Table 'properties' deleted successfully

Deleting database...
Database 'properties_db' deleted successfully

Database connection closed
